In [6]:
import jax.numpy as jnp
from jax import jit, grad
import jax
from jax.experimental import checkify
import matplotlib.pyplot as plt
from IPython.display import HTML
import sys
sys.path.append(".")

# from models.mesh_generation import (icosphere, vertex_to_polar, face_center,
#                              apply_pulsation, mesh_polar_vertices,
#                              apply_spherical_harm_pulsation, spherical_harmonic,
#                              calculate_los_rotation)

from abc import ABCMeta, abstractmethod, abstractproperty
from dataclasses import dataclass
import jax.numpy as jnp
from models.mesh_generation import icosphere
from celluloid import Camera
from spectrum import *
from .mesh_model import MeshModel

from jax.config import config
config.update("jax_enable_x64", True)
config.update("jax_debug_nans", True) 

ImportError: attempted relative import with no known parent package

In [5]:
v, f, a, c, vm = icosphere(1000)

In [7]:
phases = jnp.linspace(0, 8*jnp.pi, 100)
timestamps = jnp.linspace(2015., 2020., 100)

In [10]:
v.shape

(1272, 3)

In [11]:
verts = jnp.repeat(v[jnp.newaxis, :, :], phases.shape[0], axis=0)

In [12]:
verts.shape

(100, 1272, 3)

In [13]:
los_vector = jnp.array([1., 0., 0.])

In [19]:
jnp.linalg.norm(verts, axis=2)

Array([[1., 1., 1., ..., 0., 0., 0.],
       [1., 1., 1., ..., 0., 0., 0.],
       [1., 1., 1., ..., 0., 0., 0.],
       ...,
       [1., 1., 1., ..., 0., 0., 0.],
       [1., 1., 1., ..., 0., 0., 0.],
       [1., 1., 1., ..., 0., 0., 0.]], dtype=float64)

In [23]:
(-1.*jnp.dot(jnp.nan_to_num(verts/(jnp.linalg.norm(verts, axis=2, keepdims=True)+1e-10)), los_vector)).shape

(100, 1272)

In [24]:
generate_frames = jax.jit(jax.vmap(calculate_los_rotation, in_axes=(None, None, None, None, 0)))

NameError: name 'calculate_los_rotation' is not defined

In [ ]:
import matplotlib.pyplot as plt
import matplotlib as mpl
import numpy as np

cmap = mpl.cm.turbo
n = 10

rotation_axis = jnp.array([0., 0., 1.])

ts = jnp.linspace(0, 10, n)
    
all_centers, all_vels, _ = generate_frames(jnp.array([0.5]), rotation_axis, c, ts)

los_vels = np.nan_to_num(np.array(all_vels))

norm = mpl.colors.Normalize(vmin=los_vels.min(), vmax=los_vels.max())

In [ ]:
los_vels[i].shape

In [ ]:
import matplotlib
fig = plt.figure(figsize=(10, 10))
spec = fig.add_gridspec(10, 10)
ax1 = fig.add_subplot(spec[:, :], projection='3d') #spec[:9], 
camera = Camera(fig)

ax1.set_xlim3d(-1.5, 1.5);
ax1.set_ylim3d(-1.5, 1.5);
ax1.set_ylim3d(-1.5, 1.5);

LOG_WAVELENGTHS = jnp.linspace(jnp.log10(5850), jnp.log10(5925), 1000)

x, y, z = np.array([[0,0,0],[0,0,0],[0,0,0]])
qu, qv, qw = np.array([[1.,0,0], [0,1., 0],[0,0,1.]])
i = 5
sv = all_centers[i]

ax1.quiver(0., -3., 0., *los, color='red', linewidth=3.)
ax1.quiver(0., 0., 0., *rotation_axis, color='black', linewidth=3.)
p = ax1.scatter(sv[:, 0], sv[:, 1], sv[:, 2], c=r[i]*los_vels[i], cmap='turbo')
plt.colorbar(p);

In [ ]:
LOG_WAVELENGTHS = jnp.linspace(jnp.log10(5850), jnp.log10(5925), 1000)
#atmo = spectrum_flash_sum(LOG_WAVELENGTHS, all_areas[i][mu_mask].reshape((-1, 1)), mus[mu_mask], los_vels[i][mu_mask], 0.5*jnp.ones(20,), 256)

In [ ]:
from tqdm import tqdm

In [ ]:
fig = plt.figure(figsize=(10, 15))
spec = fig.add_gridspec(15, 10)
ax1 = fig.add_subplot(spec[:9, :9], projection='3d') #spec[:9], 
ax2 = fig.add_subplot(spec[:9, 9])
# ax3 = fig.add_subplot(spec[9, 1:-1])
ax4 = fig.add_subplot(spec[10:, 1:-1])
camera = Camera(fig)

ax1.set_xlim3d(-1.5, 1.5);
ax1.set_ylim3d(-1.5, 1.5);
ax1.set_ylim3d(-1.5, 1.5);

LOG_WAVELENGTHS = jnp.linspace(jnp.log10(5885), jnp.log10(5895), 500)

x, y, z = np.array([[0,0,0],[0,0,0],[0,0,0]])
qu, qv, qw = np.array([[1.,0,0], [0,1., 0],[0,0,1.]])

for i, (sv, v) in tqdm(enumerate(zip(all_centers, all_vels))):

    ax1.quiver(0., -3., 0., *los, color='red', linewidth=3.)
    mus = -1.*jnp.nan_to_num(jnp.dot(sv/jnp.linalg.norm(sv, axis=1).reshape((-1, 1)), los))
    p = ax1.scatter(sv[:, 0], sv[:, 1], sv[:, 2], c=los_vels[i]*r[i], cmap=cmap, norm=norm);

    rot_mat = rotation_matrix(rotation_axis, 0.5*ts[i])
    rqu = jnp.matmul(qu, rot_mat)
    rqv = jnp.matmul(qv, rot_mat)
    rqw = jnp.matmul(qw, rot_mat)

    ax1.quiver(x, y, z, rqu, rqv, rqw, arrow_length_ratio=0.1, color="black", linewidth=1.)
    
    ax1.set_xlim(-1.5, 1.5)
    ax1.set_ylim(-1.5, 1.5)
    ax1.set_zlim(-1.5, 1.5)
    
    mu_mask = mus>0
    
    atmo1 = spectrum_flash_sum(LOG_WAVELENGTHS,
                              (mus[mu_mask]*a[mu_mask]).reshape((-1, 1)),
                              mus[mu_mask],
                              5*los_vels[i][mu_mask]*r[i][mu_mask], 0.5*jnp.ones(20,), 256)
    atmo2 = spectrum_flash_sum(LOG_WAVELENGTHS,
                              (mus[mu_mask]*a[mu_mask]).reshape((-1, 1)),
                              mus[mu_mask],
                              20*los_vels[i][mu_mask]*r[i][mu_mask], 0.5*jnp.ones(20,), 256)
    atmo3 = spectrum_flash_sum(LOG_WAVELENGTHS,
                              (mus[mu_mask]*a[mu_mask]).reshape((-1, 1)),
                              mus[mu_mask],
                              50*los_vels[i][mu_mask]*r[i][mu_mask], 0.5*jnp.ones(20,), 256)
    
    ax4.plot(10**LOG_WAVELENGTHS, atmo1[0]/atmo1[1], color='green', zorder=1, label='5 km/s')
    ax4.plot(10**LOG_WAVELENGTHS, atmo2[0]/atmo2[1], color='red', zorder=1, label='20 km/s')
    ax4.plot(10**LOG_WAVELENGTHS, atmo3[0]/atmo3[1], color='royalblue', zorder=1, label='50 km/s')
    
    ax4.set_xlabel('Wavelength [$\AA$]')
    
    camera.snap()

cbar = fig.colorbar(mpl.cm.ScalarMappable(norm=norm, cmap=cmap), cax=ax2)
cbar.set_label('LOS velocity')

anim = camera.animate(blit=True)
anim.save('test_w_transformer_spectrum.gif')